# <center>LightGBM hyperparameter tuning</center>

# 1. Choosing and importing right libraries
[Optuna](https://optuna.org/) is one of the best and easiest to use libraries for optimization. Its' API allows for great flexibility when it comes to creating, stopping and editing trials making it a very important tool for Kaggle contests. Since Optuna has a specialized API for XGBoost models, we will use it to speed up the search.

In [1]:
!pip install optuna-integration

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.4/93.4 kB 2.3 MB/s eta 0:00:00


In [2]:
!pip install lightgbm --upgrade

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 38.2 MB/s eta 0:00:00
  Attempting uninstall: lightgbm
    Found existing installation: lightgbm 4.2.0
    Uninstalling lightgbm-4.2.0:
      Successfully uninstalled lightgbm-4.2.0


In [3]:
import warnings
import gc
import importlib
import os

warnings.filterwarnings("ignore", category=FutureWarning, module="seaborn")

import pandas as pd
pd.set_option('display.float_format', lambda x: '%.3f' % x)

from sklearn.metrics import matthews_corrcoef

import lightgbm as lgb

from multiprocessing import cpu_count

import optuna
opt_int = importlib.import_module("optuna_integration")

import pickle

import numpy as np

In [4]:
RAND = 42

In [5]:
import shutil
src_path = r"/kaggle/input/ps4e7-lgbmclassifier-tuner/lgbm.db"
dst_path = r"/kaggle/working/lgbm.db"

try:
    shutil.copy(src_path, dst_path)
    print('Copied')
except FileNotFoundError:
    print("lgbm.db not found!")

lgbm.db not found!


# 2. Preparing data for LightGBM model
We will use LightGBM native API as it allows for better flexibility with Callbacks and early stopping which are crucial for the development. In order to do that, we need to prepare our data and turn it into DMatrix with categorical features.

In [6]:
X_train = pd.read_parquet('/kaggle/input/ps4e8-data-eng/train.parquet')
y_train = X_train.pop('class')

df_train = lgb.Dataset(X_train, label=y_train)

X_test = pd.read_parquet('/kaggle/input/ps4e8-data-eng/test.parquet')
y_test = X_test.pop('class')

df_test = lgb.Dataset(X_test, label=y_test, reference=df_train)

X_train.head()

,cap-diameter,cap-shape,cap-surface,cap-color,does-bruise-or-bleed,gill-attachment,gill-spacing,gill-color,stem-height,stem-width,stem-root,stem-surface,stem-color,veil-type,veil-color,has-ring,ring-type,spore-print-color,habitat,season
id,,,,,,,,,,,,,,,,,,,,
2413129,16.160,0,6,1,0,4,-1,0,16.890,17.660,3,0,0,0,1,1,5,-1,0,2
1910266,3.420,4,-1,10,0,5,2,5,3.620,15.310,-1,3,8,-1,-1,0,0,-1,0,2
1213509,8.110,0,1,8,0,-1,0,0,11.250,15.030,-1,-1,0,0,1,1,5,-1,0,2
387249,6.670,5,10,7,1,1,0,6,7.320,16.530,-1,-1,0,-1,-1,0,0,-1,0,0
2498558,12.230,0,2,5,1,4,-1,0,10.180,16.800,-1,-1,0,-1,-1,1,-1,-1,0,2


In [7]:
train_cols = X_train.columns
train_cols

Index(['cap-diameter', 'cap-shape', 'cap-surface', 'cap-color',
       'does-bruise-or-bleed', 'gill-attachment', 'gill-spacing', 'gill-color',
       'stem-height', 'stem-width', 'stem-root', 'stem-surface', 'stem-color',
       'veil-type', 'veil-color', 'has-ring', 'ring-type', 'spore-print-color',
       'habitat', 'season'],
      dtype='object')

In [8]:
X_train.shape

(2493556, 20)

In [9]:
X_train.dtypes

cap-diameter            float64
cap-shape                 int64
cap-surface               int64
cap-color                 int64
does-bruise-or-bleed      int64
gill-attachment           int64
gill-spacing              int64
gill-color                int64
stem-height             float64
stem-width              float64
stem-root                 int64
stem-surface              int64
stem-color                int64
veil-type                 int64
veil-color                int64
has-ring                  int64
ring-type                 int64
spore-print-color         int64
habitat                   int64
season                    int64
dtype: object

# 3. Defining custom XGBoost metric
Since the contest uses a custom (not available in LightGBM) metric, we need to create it ourselves! The metric used is [Matthews correlation coefficient](https://en.wikipedia.org/wiki/Phi_coefficient) known in statistics as phi coefficient.

In [10]:
def mcc(preds: np.ndarray, data: lgb.Dataset):
    # # eval_name, eval_result, is_higher_better
    return 'mcc', matthews_corrcoef(data.get_label(), np.round(preds)), True

In [11]:
def objective(trial):
    params = {
        'num_leaves': trial.suggest_int('num_leaves', 10, 128),
        'max_depth': trial.suggest_int('max_depth', 3, 10),
        'learning_rate': 0.01,
        'subsample': trial.suggest_float('subsample', 0.2, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.2, 1.0),
        'reg_alpha': trial.suggest_int('reg_alpha', 1, 1000, log=True),
        'reg_lambda': trial.suggest_int('reg_lambda', 1, 1000, log=True),
        'objective': 'binary',
        'metric': 'logloss',
        'seed': RAND,
        'verbose': -1,
        'num_threads': cpu_count()
    }
    
    model = lgb.train(
        params,
        df_train,
        num_boost_round=2000,
        valid_sets=[df_test],
        feval=mcc,
        callbacks=[lgb.early_stopping(300, verbose=True, min_delta=1e-4), lgb.log_evaluation(period=20)]
    )
    
    return matthews_corrcoef(y_test, np.round(model.predict(X_test)))

# 4. Using existing Optuna study
We can reuse already existing study if it exists. This way we don't have to worry about Kaggle's 12 hour limit as we can just run this notebook multiple times each time reusing sampler and pruner from previous one. [Learn more](https://optuna.readthedocs.io/en/stable/tutorial/20_recipes/001_rdb.html).

In [12]:
study_name = 'lgbm-tune'
storage = 'sqlite:///lgbm.db'

In [13]:
try:
    sampler = pickle.load(open("/kaggle/input/ps4e7-lgbmclassifier-tuner/sampler.pkl", "rb"))
    print("Choosing saved sampler!")
except FileNotFoundError:
    sampler = None
    print("Default sampler")

Default sampler


In [14]:
try:
    pruner = pickle.load(open("/kaggle/input/ps4e7-lgbmclassifier-tuner/pruner.pkl", "rb"))
    print("Choosing saved pruner!")
except FileNotFoundError:
    pruner = pruner = optuna.pruners.HyperbandPruner()
    print("Default pruner")

Default pruner


In [15]:
study = optuna.create_study(study_name=study_name, storage=storage, direction='maximize', load_if_exists=True, sampler=sampler, pruner=pruner)
study.optimize(objective, n_trials=20)

[I 2024-08-08 10:49:33,242] A new study created in RDB with name: lgbm-tune


Training until validation scores don't improve for 300 rounds
[20]	valid_0's mcc: 0.525939
[40]	valid_0's mcc: 0.585949
[60]	valid_0's mcc: 0.66847
[80]	valid_0's mcc: 0.732165
[100]	valid_0's mcc: 0.767145
[120]	valid_0's mcc: 0.784729
[140]	valid_0's mcc: 0.799377
[160]	valid_0's mcc: 0.79945
[180]	valid_0's mcc: 0.807205
[200]	valid_0's mcc: 0.810993
[220]	valid_0's mcc: 0.814103
[240]	valid_0's mcc: 0.820454
[260]	valid_0's mcc: 0.833051
[280]	valid_0's mcc: 0.838163
[300]	valid_0's mcc: 0.844857
[320]	valid_0's mcc: 0.848629
[340]	valid_0's mcc: 0.851061
[360]	valid_0's mcc: 0.856363
[380]	valid_0's mcc: 0.869478
[400]	valid_0's mcc: 0.871615
[420]	valid_0's mcc: 0.873935
[440]	valid_0's mcc: 0.875103
[460]	valid_0's mcc: 0.876137
[480]	valid_0's mcc: 0.876769
[500]	valid_0's mcc: 0.878065
[520]	valid_0's mcc: 0.87847
[540]	valid_0's mcc: 0.878217
[560]	valid_0's mcc: 0.880812
[580]	valid_0's mcc: 0.884607
[600]	valid_0's mcc: 0.88671
[620]	valid_0's mcc: 0.887837
[640]	valid_0's 

[I 2024-08-08 11:07:00,671] Trial 0 finished with value: 0.9604127819411883 and parameters: {'num_leaves': 115, 'max_depth': 4, 'subsample': 0.7777110917680685, 'colsample_bytree': 0.971475963881673, 'reg_alpha': 715, 'reg_lambda': 420}. Best is trial 0 with value: 0.9604127819411883.


Training until validation scores don't improve for 300 rounds
[20]	valid_0's mcc: 0.454831
[40]	valid_0's mcc: 0.560945
[60]	valid_0's mcc: 0.574951
[80]	valid_0's mcc: 0.624202
[100]	valid_0's mcc: 0.645406
[120]	valid_0's mcc: 0.639195
[140]	valid_0's mcc: 0.647958
[160]	valid_0's mcc: 0.659827
[180]	valid_0's mcc: 0.684768
[200]	valid_0's mcc: 0.691742
[220]	valid_0's mcc: 0.693665
[240]	valid_0's mcc: 0.691419
[260]	valid_0's mcc: 0.68866
[280]	valid_0's mcc: 0.692034
[300]	valid_0's mcc: 0.701632
[320]	valid_0's mcc: 0.712056
[340]	valid_0's mcc: 0.731153
[360]	valid_0's mcc: 0.74003
[380]	valid_0's mcc: 0.747313
[400]	valid_0's mcc: 0.769772
[420]	valid_0's mcc: 0.773007
[440]	valid_0's mcc: 0.779016
[460]	valid_0's mcc: 0.781154
[480]	valid_0's mcc: 0.786983
[500]	valid_0's mcc: 0.793265
[520]	valid_0's mcc: 0.795029
[540]	valid_0's mcc: 0.803718
[560]	valid_0's mcc: 0.805373
[580]	valid_0's mcc: 0.807344
[600]	valid_0's mcc: 0.812373
[620]	valid_0's mcc: 0.818604
[640]	valid_0'

[I 2024-08-08 11:24:11,134] Trial 1 finished with value: 0.9128963702908467 and parameters: {'num_leaves': 12, 'max_depth': 3, 'subsample': 0.4211725054216306, 'colsample_bytree': 0.7766507288251865, 'reg_alpha': 23, 'reg_lambda': 292}. Best is trial 0 with value: 0.9604127819411883.


...

[I 2024-08-08 17:19:25,215] Trial 17 finished with value: 0.98286152081742 and parameters: {'num_leaves': 70, 'max_depth': 9, 'subsample': 0.8903600659887809, 'colsample_bytree': 0.5666651790828627, 'reg_alpha': 36, 'reg_lambda': 65}. Best is trial 16 with value: 0.9832484155231614.


Training until validation scores don't improve for 300 rounds
[20]	valid_0's mcc: 0.788047
[40]	valid_0's mcc: 0.894442
[60]	valid_0's mcc: 0.916428
[80]	valid_0's mcc: 0.932724
[100]	valid_0's mcc: 0.94257
[120]	valid_0's mcc: 0.941736
[140]	valid_0's mcc: 0.942966
[160]	valid_0's mcc: 0.944468
[180]	valid_0's mcc: 0.945015
[200]	valid_0's mcc: 0.948075
[220]	valid_0's mcc: 0.950208
[240]	valid_0's mcc: 0.951153
[260]	valid_0's mcc: 0.952868
[280]	valid_0's mcc: 0.954543
[300]	valid_0's mcc: 0.956361
[320]	valid_0's mcc: 0.957634
[340]	valid_0's mcc: 0.959288
[360]	valid_0's mcc: 0.960359
[380]	valid_0's mcc: 0.961565
[400]	valid_0's mcc: 0.962633
[420]	valid_0's mcc: 0.963542
[440]	valid_0's mcc: 0.964635
[460]	valid_0's mcc: 0.966972
[480]	valid_0's mcc: 0.969172
[500]	valid_0's mcc: 0.970128
[520]	valid_0's mcc: 0.970725
[540]	valid_0's mcc: 0.971239
[560]	valid_0's mcc: 0.971751
[580]	valid_0's mcc: 0.972006
[600]	valid_0's mcc: 0.972323
[620]	valid_0's mcc: 0.973075
[640]	valid_0

[I 2024-08-08 17:41:12,124] Trial 18 finished with value: 0.9813462460068485 and parameters: {'num_leaves': 50, 'max_depth': 7, 'subsample': 0.665566312220514, 'colsample_bytree': 0.6498363720364115, 'reg_alpha': 12, 'reg_lambda': 57}. Best is trial 16 with value: 0.9832484155231614.


Training until validation scores don't improve for 300 rounds
[20]	valid_0's mcc: 0.579892
[40]	valid_0's mcc: 0.875019
[60]	valid_0's mcc: 0.925874
[80]	valid_0's mcc: 0.941711
[100]	valid_0's mcc: 0.949874
[120]	valid_0's mcc: 0.956535
[140]	valid_0's mcc: 0.961255
[160]	valid_0's mcc: 0.962777
[180]	valid_0's mcc: 0.963332
[200]	valid_0's mcc: 0.965424
[220]	valid_0's mcc: 0.966826
[240]	valid_0's mcc: 0.967439
[260]	valid_0's mcc: 0.968038
[280]	valid_0's mcc: 0.968721
[300]	valid_0's mcc: 0.969241
[320]	valid_0's mcc: 0.969668
[340]	valid_0's mcc: 0.970089
[360]	valid_0's mcc: 0.970196
[380]	valid_0's mcc: 0.970505
[400]	valid_0's mcc: 0.97096
[420]	valid_0's mcc: 0.971225
[440]	valid_0's mcc: 0.971437
[460]	valid_0's mcc: 0.97182
[480]	valid_0's mcc: 0.972272
[500]	valid_0's mcc: 0.972799
[520]	valid_0's mcc: 0.97388
[540]	valid_0's mcc: 0.974178
[560]	valid_0's mcc: 0.974523
[580]	valid_0's mcc: 0.974746
[600]	valid_0's mcc: 0.974998
[620]	valid_0's mcc: 0.975177
[640]	valid_0's

[I 2024-08-08 18:03:48,601] Trial 19 finished with value: 0.9802511914362402 and parameters: {'num_leaves': 78, 'max_depth': 10, 'subsample': 0.8685413593045525, 'colsample_bytree': 0.3134558519255893, 'reg_alpha': 335, 'reg_lambda': 77}. Best is trial 16 with value: 0.9832484155231614.


# 5. Saving sampler and pruner
In order to be able to easily pick up current study next time, we have to save our sampler and pruner. [Learn more](https://optuna.readthedocs.io/en/stable/tutorial/20_recipes/001_rdb.html#resume-study).

In [16]:
with open("sampler.pkl", "wb") as fout:
    pickle.dump(study.sampler, fout)
    print("Sampler saved!")
    
with open("pruner.pkl", "wb") as fout:
    pickle.dump(study.pruner, fout)
    print("Pruner saved!")

Sampler saved!
Pruner saved!


In [19]:
print(f"Best score: {study.best_value}")
print(study.best_params)

Best score: 0.9832484155231614
{'num_leaves': 73, 'max_depth': 9, 'subsample': 0.8704060497676991, 'colsample_bytree': 0.32575454125826914, 'reg_alpha': 47, 'reg_lambda': 66}
